In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

all_columns = ["timestamps_UTC",
               "lat","lon",
               
               "RS_E_OilPress_PC1","RS_E_OilPress_PC2",

               "RS_E_RPM_PC1","RS_E_RPM_PC2",

               "RS_E_InAirTemp_PC1","RS_E_InAirTemp_PC2",
               "RS_E_WatTemp_PC1","RS_E_WatTemp_PC2",
               "RS_T_OilTemp_PC1","RS_T_OilTemp_PC2"]



In [3]:
col_types = {"mapped_veh_id": np.int32,
            #"RS_E_OilPress_PC1": np.int32,
            #"RS_E_OilPress_PC2": np.int32,
            #"RS_E_RPM_PC1": np.int32,
            #"RS_E_RPM_PC2": np.int32,
            #"RS_E_InAirTemp_PC1": np.int32,
            #"RS_E_InAirTemp_PC2": np.int32,
            #"RS_E_WatTemp_PC1": np.int32,
            #"RS_E_WatTemp_PC2": np.int32,
            #"RS_T_OilTemp_PC1": np.int32,
            #"RS_T_OilTemp_PC2": np.int32
            }
data = pd.read_csv("../data/sorted_full_data.csv", delimiter=";", index_col=0, dtype=col_types)
data

,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,RS_E_RPM_PC2,RS_E_WatTemp_PC1,RS_E_WatTemp_PC2,RS_T_OilTemp_PC1,RS_T_OilTemp_PC2
mapped_veh_id,,,,,,,,,,,,,
102,2023-01-23 07:25:08,51.017864,3.769079,17.0,18.0,210.0,210.0,858.0,839.0,78.0,80.0,71.0,79.0
102,2023-01-23 07:25:16,51.017875,3.769046,17.0,20.0,200.0,200.0,801.0,804.0,79.0,80.0,76.0,79.0
102,2023-01-23 07:25:37,51.017208,3.770179,19.0,20.0,193.0,207.0,803.0,808.0,80.0,81.0,79.0,81.0
102,2023-01-23 07:25:41,51.016916,3.771036,19.0,20.0,196.0,203.0,801.0,803.0,80.0,81.0,79.0,81.0
102,2023-01-23 07:26:10,51.016503,3.772182,19.0,21.0,200.0,203.0,795.0,807.0,80.0,82.0,79.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,2023-09-13 17:33:03,50.402693,4.450111,37.0,40.0,220.0,258.0,803.0,803.0,81.0,79.0,77.0,81.0
197,2023-09-13 17:33:58,50.401657,4.452693,37.0,38.0,224.0,307.0,843.0,941.0,80.0,78.0,77.0,80.0
197,2023-09-13 17:34:03,50.401830,4.452217,37.0,38.0,224.0,307.0,841.0,932.0,80.0,78.0,77.0,80.0


In [23]:

#_______________UAD (Univariate Anomaly Detection):
"""
    This method looks at one feature at a time to identify anomalies. If the anomalies can be detected by examining 
    individual metrics, then we can apply statistical methods to do this. Methods like:
        -Z-Score: Measures how many standard deviations a data point is from the mean. Data points with an absolute z-score above a threshold (e.g., 3) 
            are considered outliers.
        -IQR (Interquartile Range): It's the range between the first quartile and the third quartile. Data points below (1st quartile - 1.5 x IQR) or 
            above (3rd quartile + 1.5 x IQR) can be considered outliers.
"""

def z_score_anomaly_detection(chunk, column_name, threshold=2):
    """
    Detects anomalies in a given chunk based on z-score.
    The basic idea is to determine mean/standard deviation value to calculate the z-score.
    The z-score tells us how far the measured value is far from the mean value (how many standard deviations)
    If the corresponding z-score is higher than 2-3 time the standard deviation we count the measure as an anomaly.
    This technique is usefull when we can notice errors in one column.
    

    """
    
    mean = chunk[column_name].mean()
    std_dev = chunk[column_name].std()

    
    # Calculate the z-scores
    chunk['z_score'] = (chunk[column_name] - mean) / std_dev

    # Filter rows where absolute z-score is greater than the threshold
    anomalies = chunk[chunk['z_score'].abs() > threshold]
    
    return anomalies, mean, std_dev


current_all_columns = [
               "lat","lon",
               
               "RS_E_OilPress_PC1","RS_E_OilPress_PC2",

               "RS_E_RPM_PC1","RS_E_RPM_PC2",

               "RS_E_InAirTemp_PC1","RS_E_InAirTemp_PC2",
               "RS_E_WatTemp_PC1","RS_E_WatTemp_PC2",
               "RS_T_OilTemp_PC1","RS_T_OilTemp_PC2"]

# Open the file for writing
with open('../output.txt', 'w') as file:
    current_all_columns = [
        "lat", "lon",
        "RS_E_OilPress_PC1", "RS_E_OilPress_PC2",
        "RS_E_RPM_PC1", "RS_E_RPM_PC2",
        "RS_E_InAirTemp_PC1", "RS_E_InAirTemp_PC2",
        "RS_E_WatTemp_PC1", "RS_E_WatTemp_PC2",
        "RS_T_OilTemp_PC1", "RS_T_OilTemp_PC2"
    ]

    for name in current_all_columns:
        anomalies, mean, std_dev = z_score_anomaly_detection(data, name)
        file.write(f"column_name ({name}) with a threshold ({2}): \n\t-The mean value: {mean} \n\t-The standart deviation: {std_dev}\n")
    
        file.write(f"Here's the lengths: \n\t-data: {len(data)} \n\t-anomalies: {len(anomalies)} \n\t-ratio: {(len(anomalies)/len(data)):.2f}%\n")
        file.write('______'*10 + "\n\n")

In [26]:


#_______________MAD (Multivariate Anomaly Detection):
"""
This looks at combinations of features to detect anomalies.

    -PCA (Principal Component Analysis): Transform data into principal components and then analyze residuals (difference between original and reconstructed data) to detect 
        anomalies.
    -Clustering (e.g., K-means): Data points that are far from cluster centers can be considered anomalies.


"""

#_______________TSAD (Time-Series Anomaly Detection):
"""
In the context of trains/metros, we might want to consider methods designed for such data.
    -Exponential Smoothing, ARIMA: Model the time-series and then check for data points that deviate significantly from the predicted values.
    -Moving Average: Calculate the moving average and detect points that deviate from this average by more than a predefined threshold.
"""




'\nIn the context of trains/metros, we might want to consider methods designed for such data.\n    -Exponential Smoothing, ARIMA: Model the time-series and then check for data points that deviate significantly from the predicted values.\n    -Moving Average: Calculate the moving average and detect points that deviate from this average by more than a predefined threshold.\n'